# Text Classification by using Topic model (LDA)  
2019/01/18

### Business Impact    
　Ted Talk や Medium などのテキスト、音声、動画等全てのコンテンツプラットフォームにおいてトピックカテゴリごとにユーザーからパブリッシュされたコンテンツを自動分類する役割を担っている。    
　このトピックごとの分類をもとに適切なレコメンドが可能となる。確率分布、共役事前分布、パラメータ推定の３点がかなり重要になる。   

![title](https://theintelligenceofinformation.files.wordpress.com/2016/12/topic-modeling-for-learning-analytics-researchers-lak15-tutorial-15-638.jpg)

### 【MY GOAL】
### トピックモデルを使って、ニュースタイトルを解析してカテゴリー分類する機械学習モデルを構築する

## 1.【学習/テストデータの準備】 Import Data

In [1]:
import pandas as pd

data = pd.read_csv("abcnews-date-text.csv", error_bad_lines=False)
data.head(3)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit


In [2]:
title = data[["headline_text"]]
title["index"] = title.index
print("記事数 : {}".format(len(title)))
title[:5]

記事数 : 1103663


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## 2. 【自然言語の前処理】Preprocessing for NLP

ここでは以下、5つの処理を行う。 
   
1. Tokenization 記事を文単位に。文章を単語単位に分解する作業。   
2. Removing meaningless words 3アルファベット以下の単語の除外（英語の自然言語処理でよく用いられる前処理）。  
3. Stopwords ある言語全体で頻出するため特徴を表現するものではない単語の除外。  
4. Lemmatizing 時制や形式の統一（meeting, meets, met→ meet）。   
5. Stemming 同じ語幹を持つ単語の統一。   
##### gensim(1&3)とnltk(4&5)というNLPの前処理のためのライブラリをダウンロード

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import * 
import numpy as np
np.random.seed(2018)

import nltk
# corpusをダウンロード
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/akr712/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Lemmatizingの例

In [4]:
WordNetLemmatizer().lemmatize("went", pos="v")

'go'

Stemmingの例

In [5]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational']

stemmed_words = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed word': stemmed_words})

,original word,stemmed word
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


1~5で一気に前処理（パイプライン処理）

In [6]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

元データのタイトルテキストと、前処理後のタイトルを比較する

In [7]:
tidy_text = title[title["index"]==2019].values[0][0]

word_list = []
for word in tidy_text.split(" "):
    word_list.append(word)
print("Before preprocessing : {}".format(word_list))
print("After preprocessing : {}".format(preprocess(tidy_text)))

Before preprocessing : ['hunter', 'police', 'prepare', 'for', 'potential', 'terrorism']
After preprocessing : ['hunter', 'polic', 'prepar', 'potenti', 'terror']


In [8]:
tidy_titles = title["headline_text"].map(preprocess)
tidy_titles[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## 3. 【自然言語の数値化】Bag of Words 

![title](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRecLU_sSXnDqdNI_i7xSRMWKys5VSTUoVDMDPyfWurcZZi_CR3WA)

ここからオリジナルのコーパスを作成し、ベクトル化する

In [9]:
from gensim.corpora import Dictionary 
dictionary = Dictionary(tidy_titles)

count = 0
for k, i in dictionary.iteritems():
    print(k, i)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


・ "no_below=15" 15回以下しか出現しない単語は無視します   
・ "no_above=0.5" 全部の文章の50パーセント以上に出現したワードは一般的すぎるワードとして無視します   
・ "keep_n=100000" 10万語までがコーパスに載る限界単語数

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

オリジナルコーパスに載せた前処理済みの単語を、ベクトル化（＝数値化）する

### doc2bow

In [ ]:
bow_corpus = [dictionary.doc2bow(word) for word in tidy_titles]
bow_corpus[99999]

[(586, 1), (643, 1), (4235, 1), (11110, 1)]

オリジナルコーパスに載せた単語と情報を確認する

In [ ]:
bow_title_5000 = bow_corpus[5000]

for i in range(len(bow_title_5000)):
    print("word {} (\"{}\") appears {} time.".format(bow_title_5000[i][0], 
                                                     dictionary[bow_title_5000[i][0]], 
                                                     bow_title_5000[i][1]))

word 50 ("hop") appears 1 time.
word 85 ("defeat") appears 1 time.
word 382 ("zealand") appears 1 time.
word 2960 ("lanka") appears 1 time.


### 【重要単語評価】Remove More Words from Corpus by uisng TF-IDF  
TF-IDFを使ってタイトル内における単語の重要度を評価し、ベクトル化までする

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
from pprint import pprint

for word in corpus_tfidf:
    pprint(word)
    break

[(0, 0.5892908867507543),
 (1, 0.38929654337861147),
 (2, 0.4964985175717023),
 (3, 0.5046520327464028)]


## 4.【文書分類：トピックモデル】Topic Modeling with LSA, PLSA, LDA & lda2Vec

![title](https://cdn-images-1.medium.com/max/1200/1*_ZMgTsJGmR743ngZ7UxN9w.png)

### A) LDA : Latent Dirichlet Allocation   
LDAは文章中の潜在的なトピックを推定し、文章分類や、文章ベクトルの次元削減等に用いられる技術です。

#### A-1) LDA with BoW

In [ ]:
lda_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2) 

In [ ]:
for idx, topic in lda_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"elect" + 0.019*"say" + 0.018*"death" + 0.016*"hospit" + 0.016*"tasmanian" + 0.014*"labor" + 0.013*"deal" + 0.012*"china" + 0.011*"polit" + 0.011*"talk"
Topic: 1 
Words: 0.020*"nation" + 0.018*"coast" + 0.017*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"chang" + 0.015*"health" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"
Topic: 2 
Words: 0.020*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"australian" + 0.014*"turnbul" + 0.013*"price" + 0.013*"share" + 0.012*"victoria" + 0.011*"bank"
Topic: 3 
Words: 0.063*"polic" + 0.023*"crash" + 0.019*"interview" + 0.019*"miss" + 0.018*"shoot" + 0.016*"arrest" + 0.015*"investig" + 0.013*"driver" + 0.012*"search" + 0.011*"offic"
Topic: 4 
Words: 0.029*"charg" + 0.028*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face" + 0.016*"alleg" + 0.016*"brisban" + 0.015*"live" + 0.015*"jail" + 0.014*"accus"
Topic: 5 
Words: 0.035*"australia" + 0.021*"melbourn" + 0.021*"world" + 0.016*"open" + 0.015*"sydney

#### A-2) LDA with TF-IDF

In [ ]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4) 

In [ ]:
for idx, topic in lda_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

### B) LSI : Latent Semantic Indexing    
LSIは、日本語では潜在的意味解析と呼ばれ、非常に端的にこの技術をまとめると、「自動車」「車」等、同じ意味をもつ単語をうまくグルーピングして、文章中の情報量を凝縮して、要点を強めようという圧縮技術です。

## 5.【モデルの比較・評価・改善】Validation for models and Improvement

In [ ]:
tidy_titles[3000]

In [ ]:
for index, score in sorted(lda_bow[tidy_titles[3000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_bow.print_topic(index, 10)))

In [ ]:
for index, score in sorted(lda_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_tfidf.print_topic(index, 10)))

In [ ]:
test_title = "How Artificial Interigence change plitics in United States"
bow_vector = dictionary.doc2bow(preprocess(test_title))

for index, score in sorted(lda_bow[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_bow.print_topic(index, 5)))

## Conclusion

![title](https://www.tidytextmining.com/images/tidyflow-ch-6.png)

・尤度：観察データの下での仮説のもっともらしさ   
・最尤推定：事前確率を使用せずに尤度のみでパラメータを推定   
・ベイズ推定：今取得したデータだけでなく、過去の推定結果や経験に基づく事前確率も使用してパラメータを推定   